# Play with Pytorch Basic Function

In [6]:
import torch

In [7]:
x = torch.ones(3,dtype=torch.double)
y = torch.empty(3,dtype=torch.double)
m = torch.rand(3,3,dtype=torch.double)

In [24]:
x,y,m

(tensor([1., 1., 1.], dtype=torch.float64),
 tensor([5.0031e-310, 5.0031e-310, 3.9525e-322], dtype=torch.float64),
 tensor([[0.4869, 0.0831, 0.5559],
         [0.0138, 0.6059, 0.3431],
         [0.2372, 0.5979, 0.2361]], dtype=torch.float64))

In [22]:
z = x+y
z = torch.add(x,y)
# z = torch.mul(x,y) #inplace y_ multiplication
y.add_(x) # inplace y_ addition
# y.mul_(x)
print(z)
print(y)

tensor([1., 1., 1.], dtype=torch.float64)
tensor([1., 1., 1.], dtype=torch.float64)


In [15]:
z,y[0]

(tensor([1., 1., 1.], dtype=torch.float64),
 tensor(1.2214e-313, dtype=torch.float64))

# Build Neural Network in Pytorch

Ref : https://www.youtube.com/watch?v=mozBidd58VQ

In [17]:
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### Get Data

In [9]:
train = datasets.MNIST(root="data",download=True,train=True,transform=ToTensor())

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:00<00:00, 11248419.63it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 1267905.52it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:00<00:00, 9441412.63it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 7381065.00it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [10]:
train

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [11]:
dataset = DataLoader(train,32)

In [36]:
train

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

# Refer this page for calculation : https://medium.com/@muhammadshoaibali/flattening-cnn-layers-for-neural-network-694a232eda6a

In [24]:
class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1,32,(3,3)),
        nn.ReLU(),
        nn.Conv2d(32,64,(3,3)),
        nn.ReLU(),
        nn.Conv2d(64,64,(3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6),10)


      )
    """
          Q : how did he got 64*(28-6)*(28-6) and he did not use max pool , is that okay?
          This is because he didn't use padding on any of its conv layer, in general the formula is ⌊(n + 2p - k)/s⌋+1 where n=Image size, p=padding, k=Kernel_size, s=stride.   So in his case
              image size = 28*28(n*n)
              For Conv1
              n = 28, p=0 which is the default value, k=3, s=1 which is also the default value,
              ⌊(n + 2p - k)/s⌋+1 = ⌊(28 + 2(0) - 3)/1⌋+1 = 26
              For Conv2
              n = 26, p=0 , k=3, s=1 ,
              ⌊(n + 2p - k)/s⌋+1 = ⌊(26 + 2(0) - 3)/1⌋+1 = 24
              For Conv3
              n = 24, p=0 , k=3, s=1 ,
              ⌊(n + 2p - k)/s⌋+1 = ⌊(24 + 2(0) - 3)/1⌋+1 = 22
              After this when you feed it to the fc layer you multiply the output channel by the size of the image which is 64*22*22
              That is how he got 64*(28-6)*(28-6)

      """

  def forward(self,x):
    return self.model(x)

In [25]:
clf = ImageClassifier().to('cpu')
opt = Adam(clf.parameters(),lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [23]:
clf.forward

ImageClassifier(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=30976, out_features=10, bias=True)
  )
)

In [ ]:
if __name__ == "__main__":
  for epoch in range(10):
    for batch in dataset:
      X,y = batch
      yhat = clf(X)
      loss = loss_fn(yhat,y)
      opt.zero_grad()
      loss.backward()
      opt.step()

    print(f"EPoch:{epoch} loss is {loss.item()}")

EPoch:0 loss is 0.05499428138136864
EPoch:1 loss is 0.0026581543497741222
EPoch:2 loss is 0.0003981966874562204
EPoch:3 loss is 2.9311146136024036e-05
EPoch:4 loss is 0.0003539360477589071
EPoch:5 loss is 2.0824195416935254e-06
